In [39]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_path = "../models/ner"
tokenizer_path = "../models/tokenizer"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

id2label = model.config.id2label
id2label


{0: 'O', 1: 'B-MOUNTAIN', 2: 'I-MOUNTAIN'}

In [31]:
def predict(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model(**tokens)

    probs = torch.softmax(outputs.logits, dim=2)
    predictions = torch.argmax(probs, dim=2)

    for token_id, pred_id, prob in zip(tokens["input_ids"][0], predictions[0], probs[0]):
        token = tokenizer.convert_ids_to_tokens([token_id])[0]
        label = id2label[pred_id.item()]
        confidence = prob[pred_id].item()
        print(f"{token:<15} -> {label:<10} ({confidence:.2f})")

# Prediction

In [34]:
predict("Everest is the highest mountain in the world.")

[CLS]           -> O          (1.00)
Everest         -> B-MOUNTAIN (0.99)
is              -> O          (1.00)
the             -> O          (1.00)
highest         -> O          (1.00)
mountain        -> O          (1.00)
in              -> O          (1.00)
the             -> O          (1.00)
world           -> O          (1.00)
.               -> O          (1.00)
[SEP]           -> O          (1.00)


Below, we can see that the model correctly identified where the B-MOUNTAIN entity begins and where it continues.


In [35]:
predict("The symbol of Japan is Mount Fuji")

[CLS]           -> O          (0.99)
The             -> O          (1.00)
symbol          -> O          (1.00)
of              -> O          (1.00)
Japan           -> O          (1.00)
is              -> O          (1.00)
Mount           -> B-MOUNTAIN (0.85)
Fuji            -> I-MOUNTAIN (0.62)
[SEP]           -> O          (1.00)


The “##” indicates that the tokenizer split the word “Matterhorn” into subword token.Moreover, we can see that it recognizes the entire word as a single tag B-MOUNTAIN.



In [19]:
predict("Matterhorn is one of the most famous mountains in the Alps.")

[CLS]           -> O          (1.00)
Matter          -> B-MOUNTAIN (0.99)
##horn          -> B-MOUNTAIN (0.99)
is              -> O          (1.00)
one             -> O          (1.00)
of              -> O          (1.00)
the             -> O          (1.00)
most            -> O          (1.00)
famous          -> O          (1.00)
mountains       -> O          (1.00)
in              -> O          (1.00)
the             -> O          (1.00)
Alps            -> O          (1.00)
.               -> O          (1.00)
[SEP]           -> O          (1.00)


# Edge cases

The case of the text affects the result because the model was trained on cased data it distinguishes between uppercase and lowercase letters. When "Everest" is written as "everest" the model fails to recognize it as a mountain name and classifies it as a regular word instead.


In [20]:
predict("everest is beautiful during sunrise.")

[CLS]           -> O          (1.00)
ever            -> O          (1.00)
##est           -> O          (1.00)
is              -> O          (1.00)
beautiful       -> O          (1.00)
during          -> O          (1.00)
sunrise         -> O          (1.00)
.               -> O          (1.00)
[SEP]           -> O          (1.00)


Here, "Mount" isn’t tagged as a mountain name since it’s used in a general context, showing the model distinguishes common words from entities.


In [21]:
predict("Mount is a common word, but not always part of a mountain name.")

[CLS]           -> O          (1.00)
Mount           -> O          (0.85)
is              -> O          (1.00)
a               -> O          (1.00)
common          -> O          (1.00)
word            -> O          (1.00)
,               -> O          (1.00)
but             -> O          (1.00)
not             -> O          (1.00)
always          -> O          (1.00)
part            -> O          (1.00)
of              -> O          (1.00)
a               -> O          (1.00)
mountain        -> O          (1.00)
name            -> O          (1.00)
.               -> O          (1.00)
[SEP]           -> O          (1.00)


Here, an interesting situation occurred: while "Everest" received the correct B-MOUNTAIN tag, "Fuji" was labeled as I-MOUNTAIN - a continuation tag. It’s unclear whether this continuation refers to "Everest" or "Mount". Moreover, "Mount* itself was tagged as O (not an entity), but with a relatively lower confidence of 0.88 compared to 1.00 for other tokens, indicating some uncertainty in the model’s prediction.


In [38]:
predict("everest and Mount Fuji are beautiful.")

[CLS]           -> O          (1.00)
Everest         -> B-MOUNTAIN (0.79)
and             -> O          (1.00)
Mount           -> O          (0.88)
Fuji            -> I-MOUNTAIN (0.97)
are             -> O          (1.00)
beautiful       -> O          (1.00)
.               -> O          (1.00)
[SEP]           -> O          (1.00)


In this case, "Mount Fuji" was not recognized as an entity at all. This is likely due to the training dataset, where mountain names were consistently written with capital letters and there were no examples containing just "Fuji" in lowercase.


In [29]:
predict("Everest and mount Fuji are beautiful.")

[CLS]           -> O          (1.00)
Everest         -> B-MOUNTAIN (0.89)
and             -> O          (1.00)
mount           -> O          (1.00)
Fuji            -> O          (0.99)
are             -> O          (1.00)
beautiful       -> O          (1.00)
.               -> O          (1.00)
[SEP]           -> O          (1.00)
